# Step 4: Fit Barnacle model to your data

Use this notebook to fit the Barnacle model to your normalized data tensor. Fitting Barnacle to data requires tuning two model parameters: 
1. `R` -- the number of components
1. `lambda` -- the sparsity parameter

There are many methods for fitting model parameters. The cross-validated parameter search strategy here is the method used to fit Barnacle to metatranscriptomic data in the [original Barnacle manuscript](https://doi.org/10.1101/2024.07.15.603627). This strategy aims to reduce resource costs by fitting `R` first and then `lambda`, rather than both parameters simultaneously. It also depends on sample replicates for performing cross validation. If your data does not have sample replicates, you might instead consider trying split-half analysis for parameter selection, but this method is not supported by this notebook.

Please refer to the notebook [3-tensorize-data.ipynb](https://github.com/blasks/barnacle-boilerplate/blob/main/3-tensorize-data.ipynb) for proper formatting of your input data tensor. Note that in order to facilitate bootstrapping, sample ID and replicate ID are combined into a unique identifier called `'sample_replicate_id'` (how creative). This will be the name of the third mode of your tensor. The sample ID and replicate ID information is preserved in separate metadata arrays in the dataset. The script will use this information to shuffle replicates between bootstraps, which enables more robust parameter selection, and confidence intervals in the final model.

In [ ]:
# imports

import os
import pandas as pd
import numpy as np
import seaborn as sns
import tomli_w
import warnings
import xarray as xr

from matplotlib import pyplot as plt

# suppress unhelpful warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# set color palette
sns.set_palette(sns.color_palette([
    '#9B5DE5', '#FFAC69', '#00C9AE', '#FD3F92', '#0F0A0A', '#959AB1', '#FFDB66', '#FFB1CA', '#63B9FF', '#4F1DD7'
]))


In [ ]:
# USER INPUTS -- edit these variables as needed

# filepath of your input data tensor (e.g. 'directory/example-tensor.nc')
datapath = "/scratch/bgrodner/iron_ko_contigs/metat_search_results/barnacle/iron_KOs.txt-tidy_all_trim/sub_taxa_01/data-tensor.nc"

# output directory where produced files will be saved (e.g. 'data/'
outdir = (
    "/scratch/bgrodner/iron_ko_contigs/metat_search_results/barnacle/iron_KOs.txt-tidy_all_trim/sub_taxa_01"
)

In [ ]:
# check inputs

# read in data tensor
ds = xr.load_dataset(datapath)

# check input data formats
required_vars = ['data', 'sample_id', 'replicate_id', 'sample_replicate_id']
for var in required_vars:
    if var not in list(ds.variables):
        raise Exception(f"Tensor missing variable '{var}'. See `3-tensorize-data.ipynb` for proper formatting of input tensor dataset.")
if input(f'Found tensor with the following modes and dimensions:\n{dict(ds.sizes)}\nIs this correct? (Y/N):').strip().lower() == 'n':
    raise Exception(f'Please double check input tensor dataset.')
else:
    modes = list(ds.data.coords)

# check output directory
outdir = f"{outdir}/barnacle"
if not os.path.isdir(outdir):
    os.makedirs(outdir)

# check sparsity modes
sparse_modes = int(input(f'How many modes will sparsity be applied to? (Enter 0/1/2/3, default is 1):'))
if sparse_modes == 1:
    if input(f'Sparsity will be applied to mode 1-{modes[0]}\nIs this correct? (Y/N):').strip().lower() == 'n':
        raise Exception('Sparsity constraint should be applied to the first mode in the tensor. Please rearrange input tensor.')
elif sparse_modes == 2:
    if input(f'Sparsity will be applied to modes 1-{modes[0]} and 2-{modes[1]}\nIs this correct? (Y/N):').strip().lower() == 'n':
        raise Exception('Sparsity constraints should be applied to first and second modes in the tensor. Please rearrange input tensor.')

ds

In [ ]:
import numpy as np
sample_names = ds.sample_id.to_numpy()
np.all(sample_names[:-1] <= sample_names[1:])
names, counts = np.unique(sample_names, return_counts=True)
counts.max()

### Part A: Identifying optimal rank

In this step you will identify the optimal rank for your model. The rank dictates the number of components in the model. If you will be using Barnacle for cluster analysis, the rank directly corresponds to the number of clusters output by the model.

The optimal value of rank can be identified via cross-validation. You will fit a series of models with different rank values, and then compare how well these models reproduce held out replicates from your dataset. To isolate the rank parameter from the sparsity parameter, this first cross-validated parameter search is done without applying any sparsity to the model.

Identifying the optimal rank requires the following steps:
1. Enter a list of rank values you would like to assess. A good starting range might be 1 - 30 in intervals of 5.
1. Run the `grid-search.py` script using the config file generated by this notebook (`rank-search.toml`).
    - You should run this script from the command line, outside of this notebook.
    - This step can last anywhere from minutes to days, or more, depending on the size of your data, the rank values you are testing, and the specs of your computer system. If it is taking a long time to run, consider testing a smaller set of rank values, running fewer bootstraps, running on a more powerful computer, or distilling your data down to a subset you are most interested in analyzing.
1. Look for a minimum in the cross-validated sum of squared errors (SSE).
    - If you do not see a minimum, you may need to repeat these steps, testing a broader range of rank values.


In [ ]:
# USER INPUTS -- edit these variables as needed

# list values of R (rank) to test
ranks = [1,5,10,20,40,80, 160]

# list which modes of the tensor model should be constrained to be non-negative (modes are 0-indexed)
nonneg_modes = [1, 2] # default is [1, 2] (i.e. first mode unconstrained, other two modes must be nonnegative values)

# enter random seed (integer)
seed = 9481


In [ ]:
# build config file

# max number of replicates
_, counts = np.unique(ds.sample_id.data, return_counts=True)
max_rep = np.max(counts)
reps = [str(r) for r in range(max_rep)]
# replst = ['A','B','C','D','E','F','G']
# reps = replst[:max_rep]

# config file structure
config = {
    "grid": {
        "ranks": ranks,
        "lambdas": [
            [0.0, 0.0, 0.0],
        ],
    },
    "params": {
        "nonneg_modes": nonneg_modes,
        "tol": 0.00001,
        "n_iter_max": 10000,
        "n_initializations": 5,
    },
    "script": {
        "input": datapath,
        "outdir": f"{outdir}/fitting",
        "n_bootstraps": 10,
        "replicates": reps,
        "max_processes": 80,
        "seed": seed,
    },
}

# function to save config as toml file
def save_toml(config, filename="config.toml"):
    with open(filename, "wb") as f:
        tomli_w.dump(config, f)
    print(f"TOML file '{filename}' created successfully.")

# save and display config toml file
fn_out = f"{outdir}/rank-search.toml"
save_toml(config, filename=fn_out)
print(f"\n{tomli_w.dumps(config)}")

# print command to initiate rank search
print(
    f"""Run the following command from the command line:

    poetry run python grid-search.py {fn_out}
    """
)

#### Initialize cross-valiated rank search

Run the above command from your computer's command line. This step may take anywhere from minutes to days to complete. Once it has successfully completed, it should have produced a number of directories and files within your output directory, including the following csv file that contains the cross-validated SSE data:

`fitting/cv_data.csv`

You can then run the following code to look for a minimum in the cross-validated SSE data. 

In [ ]:
rank_df

In [ ]:
# visualize the result of the cross validation run

# read in cross-validation data
rank_df = pd.read_csv(f"{outdir}/fitting/cv_data.csv")
# down-select to just cross-validation data with lambda=0
rank_df = rank_df[(rank_df['modeled_replicate'] != rank_df['comparison_replicate']) & rank_df['lambda'].eq(0.)].reset_index(drop=True)
rank_df['Replicate Pair'] = rank_df['modeled_replicate'].astype(str) + ', ' + rank_df['comparison_replicate'].astype(str)
rank_df['Bootstrap'] = rank_df.bootstrap_id.astype(str)

# calculate minimum average cross-validated SSE
avg_cv_sse = rank_df.groupby('rank').sse.mean()
rank_min_sse = int(avg_cv_sse.idxmin())
print(f'The minimum average cross-validated SSE was {avg_cv_sse.min():.3}, acheived with R={rank_min_sse}.')
# if rank_min_sse == rank_df['rank'].min():
#     if rank_min_sse == 1:
#         print('This may indicate a high level of noise in your data. You may need to reconsider preprocessing and normalization.')
#     else:
#         print('Please re-run the cross-validated rank search to include lower values of rank.')
# elif rank_min_sse == rank_df['rank'].max():
#     print('Please re-run the cross-validated rank search to include higher values of rank.')
# else:
#     print(f'This indicates an optimal rank value of {optimal_rank}. Proceed to select the optimal value of lambda.')

# plot cross-validated SSE as a function of rank
fig, axis = plt.subplots(figsize=(10,5))
sns.scatterplot(data=rank_df, x='rank', y='sse', hue='Bootstrap', style='Replicate Pair', ax=axis, legend=True)
sns.lineplot(data=rank_df, x='rank', y='sse', color=sns.color_palette()[4], errorbar='sd', ax=axis)
axis.legend(loc='center left', bbox_to_anchor=(1,0.5));
axis.set(xlabel='Rank (Number of Components)', ylabel='Cross-Validated SSE'); 


In [ ]:

rank_df['mean_cluster_size'] = (1 - rank_df['mode0_factor_sparsity']) * ds.sizes[modes[0]]
summary_df = rank_df.groupby('rank')[[ 'sse', 'fms', 'mean_cluster_size', 'bootstrap_id']].agg(
    sse=('sse', 'mean'), 
    sse_sem=('sse', 'sem'), 
    fms=('fms', 'mean'), 
    fms_sem=('fms', 'sem'), 
    mean_cluster_size=('mean_cluster_size', 'mean'),
    n_bootstraps=('bootstrap_id', pd.Series.nunique)
)
display(summary_df.sse)

### Part B: Identifying optimal sparsity

In this step you will identify the optimal sparsity for your model. Sparsity can be applied to 0, 1, 2, or all three modes of your model. The default is to apply sparsity just to the first mode, which should correspond to the variable you want to cluster (e.g. genes in the case of metatranscriptomic data). It is important that the modes with sparsity constraints are the first ones listed, so please re-orient your input data tensor if that is not the case.

The optimal sparsity parameter (lambda) will be identified with another cross-validated parameter search. You will fit a series of models with different lambda values, and then compare how well these models reproduce held out replicates from your dataset. All models in this search will be fit with rank set to the optimal value identified in Part A.

Identifying the optimal lambda requires the following steps:
1. Enter a list of lambda values you would like to assess.
    - Powers of 2 or 10 are a good idea for an initial coarse search.
    - A linear search is a good idea to fine-tune the optimal value of lambda.
1. Run the `grid-search.py` script using the config file generated by this notebook (`lambda-search.toml`).
    - You should run this script from the command line, outside of this notebook.
    - This step can last anywhere from minutes to days, or more, depending on the size of your data, the lambda values you are testing, and the specs of your computer system. However it is often faster than the rank search since sparse models are faster to fit.
1. Look for an optimum in the cross-validated data. There is no single best rule for this selection, but two rules of thumb are offered below.

In [ ]:
# USER INPUTS -- edit these variables as needed

# list values of lambda (sparsity penalty) to test
# lambdas = [0., 0.01, 0.1, 1., 10.] # powers of ten (coarse search)

# lambdas = [0., 1., 2., 4., 8., 16., 32.] 
lambdas = [0.05, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8] # powers of two
# lambdas = [0., 0.5, 1., 1.5, 2., 2.5, 3., 3.5, 4., 4.5, 5.] # linear (fine-tuned search)

# list wich modes of the tensor model should have sparsity applied to them (modes are 0-indexed)
sparsity_modes = [0] # default is [0] (i.e. sparsity applied to first mode only)

# enter optimal rank (identified in part A)
# optimal_rank = 80
optimal_rank = rank_min_sse

# enter random seed (integer)
seed = 42


In [ ]:
# build config file

# update ranks and lambdas
config['grid']['ranks'] = [optimal_rank]
config['grid']['lambdas'] = [[float(l) if i in sparsity_modes else 0.0 for i in [0, 1, 2]] for l in lambdas]

# update random seed
config['script']['seed'] = seed

# save and display config toml file
save_toml(config, filename=f"{outdir}/lambda-search.toml")
print(f"\n{tomli_w.dumps(config)}")

# print command to initiate sparsity parameter search
print(
    f"""Run the following command from the command line:

    poetry run python grid-search.py {outdir}/config-lambda-search.toml
    """
)


#### Initialize cross-valiated sparsity parameter search

Run the above command from your computer's command line. This step may take anywhere from minutes to days to complete. Once it has successfully completed, it should have produced a number of directories and files within your output directory, and should have updated the `fitting/cv_data.csv` to included cross-validated data from models with the different sparsity parameters tested. These data include:
- Sum of Squared Errors (SSE): Measures how well a model fit to one subset of your data reproduces another subset of your data held out during fitting
- Factor Match Score (FMS): Measures the similarity components between models fit to different subsets of your data

You can then run the following code to help identify the optimal lambda based on the cross-validation data. There is no single best rule for this selection, but here are two rules of thumb you could consider:
- Minimum SSE: As when selecting rank, select the lambda that corresponds to the lowest average cross-validated SSE. This heuristic tends to lead to the most conservative choice of lambda.
- Maximum FMS: FMS measures the similarity of components between models fit to different subsets of your data. When you're interested in clusters this FMS can be thought to measure the robustness and reproducibility of clusters. This heuristic tends to lead to higher values of lambda than SSE and thus sparser models.

Additionally, to account for variation in the bootstraps, you can use the "one standard error rule" (see e.g. Hastie, Tibshirani and Friedman, 2009) in combination with either the min SSE or max FMS heuristic. This convention suggests that instead of taking the true optimum (min SSE or max FMS), we take the highest value of lambda that lays within one standard error of the optimum. The second block of code calculates optimal values of lambda based on this heuristic.

In [ ]:
# calculate SSE and FMS optima and visualize the result of the cross validation run

# read in cross-validation data
lambda_df = pd.read_csv(f"{outdir}/fitting/cv_data.csv")
# down-select to just cross-validation data with rank=optimal_rank
lambda_df = lambda_df[
    (lambda_df['modeled_replicate'] != lambda_df['comparison_replicate']) & 
    (lambda_df['rank'].eq(optimal_rank))
].reset_index(drop=True)
lambda_df['Replicate Pair'] = lambda_df['modeled_replicate'] + ', ' + lambda_df['comparison_replicate']
lambda_df['Bootstrap'] = lambda_df.bootstrap_id.astype(str)
lambda_df['mean_cluster_size'] = (1 - lambda_df['mode0_factor_sparsity']) * ds.sizes[modes[0]]

# calculate min SSE and max FMS
summary_df = lambda_df.groupby('lambda')[['rank', 'sse', 'fms', 'mean_cluster_size', 'bootstrap_id']].agg(
    rank=('rank', 'mean'), 
    sse=('sse', 'mean'), 
    sse_sem=('sse', 'sem'), 
    fms=('fms', 'mean'), 
    fms_sem=('fms', 'sem'), 
    mean_cluster_size=('mean_cluster_size', 'mean'),
    n_bootstraps=('bootstrap_id', pd.Series.nunique)
)
summary_df
print(f'With rank={rank_min_sse}, the minimum mean SSE was {summary_df.sse.min():.3}, acheived with λ={summary_df.sse.idxmin()}.')
print(f'With rank={rank_min_sse}, the maximum mean FMS was {summary_df.fms.max():.3}, acheived with λ={summary_df.fms.idxmax()}.')

# find optimal lambda based on one standard error rule
print('Using the 1 standard error rule:')
# one standard error rule for SSE
lamb_sse_1se = summary_df[summary_df.sse.lt(summary_df.sse.min() + summary_df.loc[summary_df.sse.idxmin(), 'sse_sem'])].index.max()
print(f"\tThe optimal λ based on SSE is {lamb_sse_1se} (SSE={summary_df.loc[lamb_sse_1se, 'sse']:.3})")
# one standard error rule for FMS
lamb_fms_1se = summary_df[summary_df.fms.gt(summary_df.fms.max() - summary_df.loc[summary_df.fms.idxmax(), 'fms_sem'])].index.max()
print(f"\tThe optimal λ based on FMS is {lamb_fms_1se} (FMS={summary_df.loc[lamb_fms_1se, 'fms']:.3})")

# display summary data as a table
display(summary_df.reset_index())

# plot cross-validated SSE and FMS as a function of lambda
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(10, 15))
for i, metric in enumerate(['sse', 'fms', 'mean_cluster_size']):
    sns.scatterplot(data=lambda_df, x='lambda', y=metric, hue='Bootstrap', style='Replicate Pair', ax=axes[i], legend=(i==2))
    sns.lineplot(data=lambda_df, x='lambda', y=metric, color=sns.color_palette()[4], errorbar='sd', ax=axes[i])
    if i==2:
        axes[i].legend(loc='center left', bbox_to_anchor=(1, 1.7));
        axes[i].set(
            title='Mean Cluster Size vs. λ', xlabel='λ (Sparsity Coefficient)', xscale='log', 
            ylabel=f'Mean Cluster Size ({modes[0]}s)');
    else:
        axes[i].set(title=f"{metric.upper()} vs. λ", xlabel='λ (Sparsity Coefficient)', xscale='log', ylabel=metric.upper());


### Part C: Generating the final model

Once you've selected optimal rank and lambda parameters, then model fitting is done! At this point, it is fine to proceed to analysis and visualization using the optimally-parameterized models generated during the parameter grid search. However, it can be helpful to generate more bootstraps of your final model, in order to produce better estimates of model variation/robustness, and to assign confidence estimates to the composition of clusters, as well as their associations with the patterns detected in each model component. 

To generate more boostraps of your final model, run the `grid-search.py` script one more time using the config file generated below (`final-model.toml`). Unless you are running a very large number of bootstraps, this should finish running more quickly than in parts A and B.

In [ ]:
# USER INPUTS -- edit these variables as needed

# enter optimal rank (identified in part A)
optimal_rank = 100
# optimal_rank = rank_min_sse

# enter optimal lambda (identified in part B)
optimal_lambda = 1.0
# optimal_lambda = lamb_fms_1se

# enter number of bootstraps to run with optimal parameters
n_bootstraps = 100

# enter random seed (integer)
seed = 525600 

In [ ]:
# build config file

# update ranks and lambdas
config['grid']['ranks'] = [optimal_rank]
config['grid']['lambdas'] = [[float(optimal_lambda) if i in sparsity_modes else 0.0 for i in [0, 1, 2]]]

# update bootstraps and random seed
config['script']['n_bootstraps'] = n_bootstraps
config['script']['seed'] = seed

# save and display config toml file
save_toml(config, filename=f"{outdir}/final-model.toml")
print(f"\n{tomli_w.dumps(config)}")

# print command to initiate sparsity parameter search
print(
    f"""Run the following command from the command line:

    poetry run python grid-search.py {outdir}/final-model.toml
    """
)
